In [1]:
%load_ext autoreload
%autoreload 2

In [13]:
import os
import io
import six.moves.urllib as urllib
import sys
import json
import tarfile
import zipfile
from typing import List


from PIL import Image, ImageDraw2, ImageDraw, ImageFont
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf


from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

os.environ["CUDA_VISIBLE_DEVICES"] = ""
sys.path.insert(0, "../")

from mystique.initial_setups import set_graph_and_tensors
from mystique.predict_card import PredictCard
from mystique.detect_objects import ObjectDetection
from mystique.utils import plot_results, COLORS, plot_bboxes
from mystique import config
from mystique.card_layout import min_area_iou, bbox_area, iou_bbox, filter_similar_bboxes

# This is needed to display the images.
import matplotlib.pyplot as plt
from matplotlib import pylab, figure

%matplotlib inline

In [3]:
def predict_bbox(img_path) -> np.array:
    """
    Predict the bounding boxes, class label and draw the bbox
    on the original image.
    """
    img = Image.open(img_path)
    img_np = np.asarray(img)
    img_np = cv2.cvtColor(img_np, cv2.COLOR_RGB2BGR)
    img_np_copy = img_np.copy()
    object_detection = ObjectDetection()
    
    output, category = object_detection.get_objects(img_np, img)
    # google models wants ymin, xmin, ymax, xmax format.
    # output["detection_boxes"] = output["detection_boxes"][:, [1, 0, 3, 2]]
    
    _imgbuf = plot_results(img,
                 output["detection_classes"],
                 output["detection_scores"],
                 output["detection_boxes"]
                )
    return _imgbuf

In [4]:
img_path = "/home/haridas/projects/AdaptiveCards-ro/source/pic2card/app/assets/samples/8.png"
a = Image.open(predict_bbox(img_path))
img_path = "/home/haridas/projects/AdaptiveCards-ro/source/pic2card/app/assets/samples/9.png"
b = Image.open(predict_bbox(img_path))

In [ ]:
# np.random.choice(range(1000))

# Better Layout setting based on object coordinate.

In [ ]:
img_path = "/home/haridas/projects/mystique/data/templates_test_data/2.png"
img1 = Image.open(img_path)

In [ ]:
# od_result = {k : np.array(v) for k, v in json.loads(open("../od_result_100.json").read()).items()}
# img_path = "/home/haridas/projects/mystique/data/templates_test_data/1.png"
# img = Image.open(img_path)

od_result = {k : np.array(v) for k, v in json.loads(open("../od_result_16.json").read()).items()}
img_path = "/home/haridas/projects/mystique/data/train_and_test-2020-Jun-05-coco/train/16.png"
img = Image.open(img_path)

In [ ]:
plt.imshow(Image.open(plot_results(img,
             scores=od_result["detection_scores"],
             boxes=od_result["detection_boxes"],
             classes=od_result["detection_classes"])))

In [ ]:
box_df = pd.DataFrame(
    [(ind, *i) for ind, i in enumerate(od_result['detection_boxes'])],
    columns=["ind", "x1", "y1", "x2", "y2"]
)
box_df["class"] = od_result["detection_classes"]

In [ ]:
# [len(i) for i in collect_rows(bbox_list)]
bbox_list = [(int(t2), t1) for (*t1, t2) in box_df.sort_values(by=["y1", "x1"]).to_numpy()[:, 1:]]
card_layout = collect_rows(bbox_list)

In [ ]:
# min_iou(*[i[1] for i in bbox_list[:2]])
# parse_row(bbox_list)

In [ ]:
check_same_row([
    15.06196403503418,
      204.00167846679688,
      149.9774169921875,
      223.87066650390625
],
     [15.363767623901367,
      186.9206085205078,
      148.37368774414062,
      206.7216339111328])

In [ ]:
# print(print_layout(card_layout))

In [ ]:
plt.figure(figsize=(10, 15))
plt.imshow(Image.open(plot_results(img,
             scores=od_result["detection_scores"],
             boxes=od_result["detection_boxes"],
             classes=od_result["detection_classes"])).resize(img.size))